In [1]:
import ee 
from ee import batch

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()
from make_clim import make_climate

from stack_images_qgis import stacking
from mask_image import define_image
from stratitication import make_strata
from stratitication import prop_allocation
from change_no_change import suit_layer 

from add_neigh import add_neigh
from add_distance import add_distance
#from change_no_change import suit_layer 
import geemap


#lc_cnc_c = suit_layer(lc_img,'mult_steps',True,None)

#print(lc_cnc.bandNames().getInfo())
#print(lc_cnc1.bandNames().getInfo())

#################CHANGE ACCESSIBILITY

val_year = 2015
train_year = 2010
i_year = 1983

lc_img = define_image(0,0,'none',train_year,5)
lc_img_f = define_image(50,0,'tmov',train_year,5)

#fst_img = stacking(2005,2009,'mean',False,False)
fst_img = stacking(train_year-10,train_year-5,'mean',False,False)
clm_mean = stacking(train_year-10,train_year-5,'p50',True,True)
clm_lterm = stacking(i_year,train_year-5,'p50',True,True)

#clm_mean = stacking(2005,2009,'p50',True,True)
#clm_low = stacking(2005,2009,'p10',True,False)
#clm_high = stacking(2005,2009,'p90',True,False)

#clm_lterm = stacking(1983,2009,'p50',True,True)

clm_anomaly = clm_mean.subtract(clm_lterm)
bn = clm_anomaly.bandNames().getInfo()
bn = ["{}{}".format(i,str('_anom')) for i in bn]
clm_anomaly = clm_anomaly.rename(bn) 

# neig1 = add_neigh(lc_img.select('lc_t1'),3,1,True) # diameter = 3
# neig2 = add_neigh(lc_img.select('lc_t1'),9,3,True)
# neig3 = add_neigh(lc_img.select('lc_t1'),27,9,True)
# neig4 = add_neigh(lc_img.select('lc_t1'),81,27,True)
ineigh = lc_img.select('lc_t1')

for i in range(21,40,6): #163 #46
    ineigh = ineigh.addBands(add_neigh(lc_img.select('lc_t1'),i,1,True))
ineigh = ineigh.select(ineigh.bandNames().remove(str('lc_t1')).getInfo())
#baneigh = ineigh.bandNames().getInfo()

roads = ee.FeatureCollection('users/verdemuskuna/aus_roads/gis_osm_roads_free_1')
roads = roads.filter(ee.Filter.inList('fclass', ['motorway','trunk','primary','secondary','unclassified','tertiary','residential','living_street']))
rd = ee.Image(0).paint(roads,'code').updateMask(fst_img.select('cindex'))
rd = rd.updateMask(rd.neq(0))
dist_roads = rd.select('constant').fastDistanceTransform(1000).sqrt().multiply(ee.Image.pixelArea().sqrt())#//.divide(1000000)//.clip(to_clip)
dist_roads = dist_roads.rename('dist_roads').updateMask(fst_img.select('cindex').add(100))

# dist_roads = dist_roads.expression(
#   '(1 + (1/dd))',{
#   'dd': dist_roads.select(['dist_roads'])
# }).rename('dist_roads')

wways = ee.FeatureCollection('users/verdemuskuna/aus_roads/gis_osm_waterways_free_1')
wways = wways.filter(ee.Filter.inList('fclass', ['river']))
ww = ee.Image(0).paint(wways,'code').updateMask(fst_img.select('cindex'))
ww = ww.updateMask(ww.neq(0))
dist_wways = ww.select('constant').fastDistanceTransform(1000).sqrt().multiply(ee.Image.pixelArea().sqrt())#//.divide(1000000)//.clip(to_clip)
dist_wways = dist_wways.rename('dist_wways').updateMask(fst_img.select('cindex').add(100))
# dist_wways = dist_wways.expression(
#   '(1 + (1/dd))',{
#   'dd': dist_wways.select(['dist_wways'])
# }).rename('dist_wways')

proj = lc_img.projection()


#crop_tas_2010 = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_crop_test')
#urb_tas_2010 = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_urb_test')
#tas_2010 = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_test')

In [2]:
img_for_train = lc_img.addBands(fst_img).addBands(clm_mean).addBands(clm_anomaly).addBands(ineigh).addBands(dist_roads).addBands(dist_wways)#.addBands(pop)

ban1 = ['lc_dep', 'lc_t1','lc_t2','lc_t3','lc_t4','lc_t5']
ban2 = fst_img.bandNames().getInfo()
ban3 = clm_mean.bandNames().getInfo()
ban4 = clm_anomaly.bandNames().getInfo()
ban5 = ineigh.bandNames().getInfo()
ban6 = dist_roads.bandNames().getInfo()
ban7 = dist_wways.bandNames().getInfo()
#ban8 = pop.bandNames().getInfo()

bands  = ban1+ban2+ban3+ban4+ban5+ban6+ban7#+ban8
#print(img_for_train.bandNames().getInfo())
print(ban2)

['cindex', 'elevation', 'slope', 'aspect', 'landforms', 'DES_000_200_EV', 'SOC', 'BDW', 'CLY', 'SLT', 'SND', 'PH', 'AWC', 'NTO', 'PTO', 'p1', 'p5', 'p6', 'p12', 'p14', 'tenure2013', 'tenure2018']


In [3]:
llx = 144.68
lly = -36.2
urx = -36.075 
ury = 144.819  #echuca
#geoma = [[llx,lly], [llx,ury], [urx,ury], [urx,lly]]
#xMin, yMin, xMax, yMax.
geom = ee.Geometry.Rectangle([144.68,-36.2,144.819,-36.075]) #echuca
lim_temp_f = ee.FeatureCollection('users/mioash/aust_cd66states')
ucl = ee.FeatureCollection('users/mioash/SUA_2016_AUST')
vic = lim_temp_f.filterMetadata('STE', 'equals', 6)
geom = vic

pop = ee.ImageCollection("projects/sat-io/open-datasets/hrslpop").median().clip(geom)
pop = pop.unmask(0).reproject(proj.atScale(30))#.clip(geom)
pop = pop.select('b1').rename('pop')

rnm = ee.Image.random(153,'uniform').multiply(250).updateMask(lc_img.select('lc_dep').add(1)).rename('random')

img_for_train = img_for_train.select(bands).addBands(pop).addBands(rnm).clip(geom)
print(img_for_train.bandNames().getInfo())

['lc_dep', 'lc_t1', 'lc_t2', 'lc_t3', 'lc_t4', 'lc_t5', 'cindex', 'elevation', 'slope', 'aspect', 'landforms', 'DES_000_200_EV', 'SOC', 'BDW', 'CLY', 'SLT', 'SND', 'PH', 'AWC', 'NTO', 'PTO', 'p1', 'p5', 'p6', 'p12', 'p14', 'tenure2013', 'tenure2018', '0_tmin_djf_2006_p50', '1_tmin_mam_2006_p50', '2_tmin_jja_2006_p50', '3_tmin_son_2006_p50', '4_tmin_year_2006', '0_tmax_djf_2006_p50', '1_tmax_mam_2006_p50', '2_tmax_jja_2006_p50', '3_tmax_son_2006_p50', '4_tmax_year_2006', '0_tmean_djf_2006_p50', '1_tmean_mam_2006_p50', '2_tmean_jja_2006_p50', '3_tmean_son_2006_p50', '4_tmean_year_2006', '0_rain_djf_2006_p50', '1_rain_mam_2006_p50', '2_rain_jja_2006_p50', '3_rain_son_2006_p50', '4_rain_year_2006', '0_tmin_djf_2006_p50_anom', '1_tmin_mam_2006_p50_anom', '2_tmin_jja_2006_p50_anom', '3_tmin_son_2006_p50_anom', '4_tmin_year_2006_anom', '0_tmax_djf_2006_p50_anom', '1_tmax_mam_2006_p50_anom', '2_tmax_jja_2006_p50_anom', '3_tmax_son_2006_p50_anom', '4_tmax_year_2006_anom', '0_tmean_djf_2006_p50_

In [4]:
lcclasses = [0,1,2,3,4,5]
cpixels = [10,10,10,10,10]

#imgn = img_for_train
idist = img_for_train.select('lc_t1').rename('lc_temp')

for i in range(0,5):
    idist = add_distance(idist,lcclasses[i],'lc_temp',geom,cpixels[i])

idist = idist.select(idist.bandNames().remove(str('lc_t1')).getInfo())
idist = idist.select(idist.bandNames().remove(str('lc_temp')).getInfo())

nat_npa = ee.Image("users/mioash/drivers_lcaus/natural_areas/nat_npa_ready")
nat_npa = nat_npa.select('npa')
to_test = ee.Image(0).clip(geom).where(nat_npa,1)
to_test = to_test.rename('npa')

distto_npa = add_distance(to_test,1,'npa',geom,1)
distto_npa = distto_npa.rename(['npa','distto_npa'])

#idist = idist#.addBands(distto_npa)#.addBands(dist_roads).addBands(dist_wways)

img_for_train = img_for_train.addBands(distto_npa.select('distto_npa')).addBands(idist)
#print(img_for_train.bandNames().getInfo())                       

In [ ]:
Map = geemap.Map(center=[-37.74,144.93], zoom=7)

Map.add_basemap("SATELLITE")

#Map.addLayer(ee.Image(strat).clip(geom), {'bands':['strata'],'min': 100, 'max': 410, 'opacity':1}, 'strat')
Landcover_vispar = {'bands':['lc_dep'],"opacity":1,"min":0,"max":5,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}
#Map.addLayer(img_for_train, Landcover_vispar, 'Dep')
#Map.addLayer(lc_cnc.clip(geom), '', 'cnc')

Map.addLayer(lc_img, Landcover_vispar, 'Dep')

Map.addLayer(img_for_train,{'bands':['b_9'],'min':0,'max':100,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, '9')
Map.addLayer(img_for_train,{'bands':['g_15'],'min':0,'max':100,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'c9')

Map.addLayer(img_for_train,{'bands':['b_15'],'min':0,'max':100,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, '15')
Map.addLayer(img_for_train,{'bands':['b_21'],'min':0,'max':100,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, '21')
Map.addLayer(img_for_train,{'bands':['b_39'],'min':0,'max':100,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, '39')
Map.addLayer(img_for_train,{'bands':['b_45'],'min':0,'max':100,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, '45')
#Map.addLayer(img_for_train,{'bands':['random'],"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'rnm')


lc_dep = img_for_train.select('lc_dep')
#Map.addLayer(dist_roads.clip(geom), {'bands':['dist_roads'],"opacity":1,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'roads')
#Map.addLayer(crop_tas_2010, {'bands':['classification'],"opacity":1,"min":0,"max":5,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'crop')
#Map.addLayer(urb_tas_2010, {'bands':['classification'],"opacity":1,"min":0,"max":5,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'urb')
#Map.addLayer(tas_2010, {'bands':['classification'],"opacity":1,"min":0,"max":5,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'tas')

b_lc = {'bands':['lc_t1'],"opacity":1,"min":0,"max":1,"palette":["d7191c","2c7bb6"]}
#Map.addLayer(ssamples, '', 'samples',False)
#Map.addLayer(lc_cnc_c.clip(geom), b_lc, 'CNC',False)
#Map.addLayer(lc_cnc_f.clip(geom), b_lc, 'FNF',False)
#Map.addLayer(lc_cnc_g.clip(geom), b_lc, 'GNG',False)
#Map.addLayer(lc_cnc_u.clip(geom), b_lc, 'UNU',False)

Map


In [5]:
strat = make_strata(lc_img_f,6,False)

ssamples = prop_allocation(ee.Image(strat).toInt(),geom,750)
#print(ssamples.size().getInfo())

#print(lc_all.bandNames().getInfo())

In [9]:
ban8 = pop.bandNames().getInfo()
ban9 = idist.bandNames().getInfo()
ban10 = distto_npa.select('distto_npa').bandNames().getInfo()

lc_imga = img_for_train.select(['lc_dep','lc_t1','lc_t2','lc_t3','lc_t4','lc_t5'])
one_banda = lc_imga.add(1)

lc1 = one_banda.select('lc_t1').multiply(10000)#.add()
lc2 = one_banda.select('lc_t2').multiply(1000)
lc3 = one_banda.select('lc_t3').multiply(100)
lc4 = one_banda.select('lc_t4').multiply(10)
lc5 = one_banda.select('lc_t5').multiply(1)
lc_past = lc1.add(lc2).add(lc3).add(lc4).add(lc5).rename('lc_past')

img_for_train = img_for_train.addBands(lc_past)

lli = img_for_train.select(ban10+['elevation']+['slope']+['aspect']+['landforms']
                                                  + ['SOC']+ ['BDW']+ ['CLY']+ ['SLT']+ ['SND']
                                                  + ['PH']+ ['AWC']+ ['NTO']+ ['PTO']
                                                  + ['p1']+ ['p5']+ ['p6']+ ['p12'] + ['p14']
                                                  + ['tenure2013'] +ban6+ban7+ban8+['cindex']
                           +['random']+['lc_dep']+['lc_past'])
#lli = lli.addBands(lc_imga)
print(lli.bandNames().getInfo())
#lc_change = lc_imgb.addBands()#.addBands(pop)



['distto_npa', 'elevation', 'slope', 'aspect', 'landforms', 'SOC', 'BDW', 'CLY', 'SLT', 'SND', 'PH', 'AWC', 'NTO', 'PTO', 'p1', 'p5', 'p6', 'p12', 'p14', 'tenure2013', 'dist_roads', 'dist_wways', 'pop', 'cindex', 'random', 'lc_dep', 'lc_past']


In [10]:

trainingvals = lli.sampleRegions(collection= ssamples, 
                            scale= 30,
                            tileScale=4)

mtry = round(len(lli.bandNames().getInfo())*(2/3),0)

classifier = ee.Classifier.smileRandomForest(150, variablesPerSplit =mtry ,
                                              minLeafPopulation=150).setOutputMode('MULTIPROBABILITY')
#classifier2 = ee.Classifier.libsvm('Voting','C_SVC','RBF',True,None,0.5)


#Train the classifier.
trainer = classifier.train(trainingvals,'lc_dep')
#trainer2 = classifier2.train(trainingvals,'lc_dep')

classified = img_for_train.classify(trainer).toDouble()#.toByte()

classNameList = ['pcrop', 'pforest', 'pgrass', 'purban', 'pwater', 'pother']
currentProbsMultiBand = classified.arrayFlatten([classNameList])             

geometri = ee.Geometry.Rectangle([143.537,-43.89,148.965,-39.42]) #//#tas

ee.batch.Export.image.toAsset(image=currentProbsMultiBand,
                              description='tas_2010_out_multiprob',
                              assetId='users/mioash/Calderon_etal_Australian_land-cover/tas_2010_out_multiprob',scale=30,
                             region = geometri,maxPixels=1e13,crs='EPSG:4326').start()


#classified2 = img_for_train.classify(trainer2).toByte()
#Map.addLayer(classified, {'bands':['classification'],"opacity":1,"min":0,"max":5,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'Classified')
#Map.addLayer(classified2, {'bands':['classification'],"opacity":1,"min":0,"max":5,"palette":["FFEE88","55AA22","DDCC66","BB0011","5566AA","882255"]}, 'Classified2')

#print(classifier.getInfo())


In [ ]:
lc_cnc_c = suit_layer(lc_img,'binary',True,0)#c
lc_cnc_f = suit_layer(lc_img,'binary',True,1)#f
lc_cnc_g = suit_layer(lc_img,'binary',True,2)#g
lc_cnc_u = suit_layer(lc_img,'binary',True,3)#u
lc_cnc_w = suit_layer(lc_img,'binary',True,4)#w
lc_cnc_o = suit_layer(lc_img,'binary',True,5)#o

In [ ]:
lc_imgb = lc_cnc_o
lcclass = 'other'

ban8 = pop.bandNames().getInfo()
ban9 = idist.bandNames().getInfo()
ban10 = distto_npa.select('distto_npa').bandNames().getInfo()
one_band = lc_imgb.add(1)

lc1 = one_band.select('lc_t1').multiply(10000)#.add()
lc2 = one_band.select('lc_t2').multiply(1000)
lc3 = one_band.select('lc_t3').multiply(100)
lc4 = one_band.select('lc_t4').multiply(10)
lc5 = one_band.select('lc_t5').multiply(1)
lc_past = lc1.add(lc2).add(lc3).add(lc4).add(lc5).rename('lc_past')

lc_imgb = lc_imgb.select('lc_dep')
lc_imgb = lc_imgb.addBands(lc_past)

lc_change = lc_imgb.addBands(img_for_train.select(ban10+['elevation']+['slope']+['aspect']+['landforms']
                                                  + ['SOC']+ ['BDW']+ ['CLY']+ ['SLT']+ ['SND']
                                                  + ['PH']+ ['AWC']+ ['NTO']+ ['PTO']
                                                  + ['p1']+ ['p5']+ ['p6']+ ['p12'] + ['p14']
                                                  + ['tenure2013'] +ban6+ban7+ban8+['cindex']+['random']))#.addBands(pop)
#'p1', 'p5', 'p6', 'p12', 'p14'
#not use soil, distances(to 1,2,3,4,5) and neighbourhood 
#lc_change_t = lc_change.updateMask(lc_imgc)

print(lc_change.bandNames().getInfo())
trainingvals1 = lc_change.sampleRegions(
                            collection= ssamples, 
                            scale= 30,
                            tileScale=8)
mtry = round(len(lc_change.bandNames().getInfo())*(2/3),0)
#print(mtry)
classifier1 = ee.Classifier.smileRandomForest(numberOfTrees=150,
                                               variablesPerSplit =mtry ,
                                              minLeafPopulation=150).setOutputMode('PROBABILITY')
#classifier2 = ee.Classifier.libsvm('Voting','C_SVC','RBF',True,None,0.5)

#Train the classifier.
trainer1 = classifier1.train(trainingvals1,'lc_dep')
#trainer2 = classifier2.train(trainingvals,'lc_dep')

classified1 = lc_change.classify(trainer1).toDouble()

geometri = ee.Geometry.Rectangle([143.537,-43.89,148.965,-39.42]) #//#tas

ee.batch.Export.image.toAsset(image=classified1,
                              description='Tas_lc2010_anuclim_nneigh_ndist_naccess_tenure_'+lcclass,
                              assetId='users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_anuclim_nneigh_ndist_naccess_tenure_'+lcclass,scale=30,
                             region = geometri,maxPixels=1e13,crs='EPSG:4326').start()


In [ ]:
llx = 143.537
lly =  -43.89
urx = 148.965
ury = -39.42  #tas
#geometri = [[llx,lly], [llx,ury], [urx,ury], [urx,lly]]
geometri = ee.Geometry.Rectangle([143.537,-43.89,148.965,-39.42]) #//#tas

ee.batch.Export.image.toAsset(image=classified1,
                              description='Tas_lc2010_urb_test',
                              assetId='users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_urb_test',scale=30,
                             region = geometri,maxPixels=1e13,crs='EPSG:4326').start()


In [ ]:
ban9 = idist.bandNames().getInfo()
ban10 = distto_npa.select('distto_npa').bandNames().getInfo()
print(ban9)
print(ban10)

In [ ]:
#NotebookApp.iopub_data_rate_limit=1e10 

dicto = trainer1.explain()
#print('Explain:',dicto.getInfo())
 
variable_importance = ee.Feature(None, ee.Dictionary(dicto).get('importance'))
 
print(variable_importance.getInfo())

In [ ]:
lc_urb_t1 = lc_cnc_u.select('lc_t1')

lc_urb_t1 = lc_urb_t1.mask(lc_urb_t1.add(1).neq(2))

class_urb_new = classified1.updateMask(lc_urb_t1)
Map.addLayer(class_urb_new, {'bands':['classification'],"opacity":1,"min":0,"max":25,"palette":["d7191c","fdae61","ffffbf","abd9e9","2c7bb6"]}, ' New Urb masked')

#Map.addLayer(classified1.updateMask(classified1.lte(25)), {'bands':['classification'],"opacity":1,"min":0,"max":25,"palette":["d7191c","fdae61","ffffbf","abd9e9","2c7bb6"]}, 'Class < 25')